# Interpretabilidad LSTM

## Imports y funciones

In [ ]:
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as mpatch
import math
from IPython.core.display import display, HTML

# Visualización 
def pinta_strings(estados, inputs, probs=None, umbral=0.0):
    # estados: lista de arrays de estados en cada instante t.
    # inputs: caracteres guardados en formato numérico en cada instante t.
    # pinta_accept: Pinta las probs (solo valido para 3 simbolos).
    # probs: Si pinta_accept es true, probabilidades de cada simbolo (solo valido para 3 simbolos).
    
    inputs_print = ''.join([int_to_char[x] for x in inputs])
    
    # Every neuron
    
    for ixNeurona in range(len(estados[0][0])):
        if np.mean(np.abs(estados[0][:, ixNeurona])) < umbral:
            continue
        ylabel = "N" + str(ixNeurona)
        
        cadena = "<h1>" + ylabel + "</h1><div style='letter-spacing: 5px;'>"
        
        for i, simbolo in enumerate(inputs_print):
            if simbolo == '\n':
                simbolo = '/n<br>'
            if simbolo == '\t':
                simbolo = '&nbsp;&nbsp;&nbsp;&nbsp;/t'
                
            n = 255 - math.ceil(estados[i][0,ixNeurona] * 255)
            
            if estados[i][0, ixNeurona] < 0:
                color = '#' + str(hex(n))[-2:] + 'FFFF'
            else:
                color = '#FF' + str(hex(n))[-2:] + str(hex(n))[-2:]
                
            cadena += "<span style='background-color:" + color + "'>" + simbolo + "</span>"
            
        cadena += "</div>"
        display(HTML(cadena))
    

## Generación de los datos

In [ ]:
raw_text = open("xml_dataset.xml", "r").read()

In [ ]:
# Mapeo de los carácteres a int
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# Resumen de los datos cargados
n_chars = len(raw_text)
n_vocab = len(chars)
print("Longitud total de carácteres: ", n_chars)
print("Número total de carácteres distintos: ", n_vocab)

In [ ]:
# Preparación del dataset de forma secuencial
seq_length = 20
data_x = []
data_y = []

for i in range(0, n_chars - seq_length, seq_length):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    data_x.append([char_to_int[char] for char in seq_in])
    data_y.append(char_to_int[seq_out])
    
n_patterns = len(data_x)
print("Número total de patrones: ", n_patterns)

## Entrenamiento de la red

In [ ]:
lstm_neurons = 50
epochs = 50
batch_size = 30
l1reg = 0

In [ ]:
# One hot encoding de la entrada
x = np.reshape(tf.keras.utils.to_categorical(data_x,num_classes=n_vocab), (n_patterns, seq_length, n_vocab))
y = tf.keras.utils.to_categorical(data_y,num_classes=n_vocab)

# Definición de la capa de entrada
input = tf.keras.layers.Input(shape=(seq_length, n_vocab))

# Creación de la LSTM
if l1reg != 0:
    lstm1 = tf.keras.layers.LSTM(lstm_neurons,name='lstm1',return_sequences=True,
        activity_regularizer=tf.keras.regularizers.l1(l1reg))(input)
    lstm2 = tf.keras.layers.LSTM(lstm_neurons,name='lstm2',
        activity_regularizer=tf.keras.regularizers.l1(l1reg))(lstm1)
else:
    lstm1 = tf.keras.layers.LSTM(lstm_neurons,name='lstm1',return_sequences=True)(input)
    lstm2 = tf.keras.layers.LSTM(lstm_neurons,name='lstm2')(lstm1)

# Añadida la capa de salida
dense = tf.keras.layers.Dense(n_vocab, activation='softmax')(lstm2)

# Compilación del modelo
model = tf.keras.models.Model(inputs=input, outputs=[dense])
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Modelo intermedio para explorar el valor de los hidden state
hidden1_model = tf.keras.models.Model(inputs=model.input,outputs=lstm1)
hidden2_model = tf.keras.models.Model(inputs=model.input,outputs=lstm2)

# Entrenamiento de la red
model.fit(x, y, epochs=epochs, batch_size=batch_size)#, steps_per_epoch=1)#,callbacks=callbacks_list)

## Predicción de la red

In [ ]:
# Se escoge una semilla inicial aleatoria
start = np.random.randint(0, len(data_x)-1)
pattern = data_x[start]
print("Semilla inicial:")
print(''.join([int_to_char[value] for value in pattern]))

# Generación de los carácteres
probs = []
hidden_states = []
hidden_states2 = []
inputs = []

for i in range(500):
    x = np.reshape(tf.keras.utils.to_categorical(pattern,num_classes=n_vocab),
        (1, len(pattern), n_vocab))
    
    # Predicción del siguiente carácter
    prediction = model.predict(x, verbose=0)
    probs.append(prediction)
    
    # Se guardan el valor de los hidden states a interpretar
    hidden_states.append(hidden1_model.predict(x,verbose=0)[-1])
    hidden_states2.append(hidden2_model.predict(x,verbose=0))
    
    # De forma aleatoria con la probabilidad dada por la red se elige el nuevo carácter
    index = np.random.choice(range(n_vocab), p=prediction.ravel())
    
    #result = int_to_char[index]
    # Se guarda el valor para compararlo con la activación
    inputs.append(index)

    # Se añade de forma recursiva el carácter generado
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [ ]:
# ROJO: ' '
# VERDE: 'a'
# AZUL: 'b'
pinta_strings(hidden_states2, inputs, probs=probs, umbral=0)